In [ ]:
import os
import time

import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, SequentialSampler, ConcatDataset

from chord_recognition.dataset import AudioDataset
from chord_recognition.metrics import compute_eval_measures
from chord_recognition.cnn import model
from chord_recognition.predict import predict_annotations, model, device

In [ ]:
# chord_dataset = AudioDataset(audio_dir="data/robbie_williams/mp3/",
#                              ann_dir="data/robbie_williams/chordlabs/",
#                             window_size=8192, hop_length=4096)
chord_dataset = AudioDataset(audio_dir="data/queen/mp3/",
                             ann_dir="data/queen/chordlabs/",
                             window_size=8192, hop_length=4096)
queen = AudioDataset(audio_dir="data/beatles/mp3s-32k/",
                     ann_dir="data/beatles/chordlabs/",
                     window_size=8192, hop_length=4096)
chord_dataset = ConcatDataset([chord_dataset, queen])
loader_chord = DataLoader(chord_dataset, batch_size=None, num_workers=0)
sampler = SequentialSampler(chord_dataset)

In [ ]:
def estimate_chords(dataloader, model, device, batch_size=32,
                    drop_last=False):
    model.eval()  # set model to evaluation mode
    total_R = total_P = total_F = 0.0
    count = 0
    
    for idx in dataloader:
        sample = dataloader.data_source[idx]
        sample_name = sample['sample']
        chromagram = sample['chromagram']
        
        result = predict_annotations(chromagram, model, device, batch_size=batch_size)
        ann_matrix = sample['ann_matrix']
        count += 1
        
        P, R, F1, TP, FP, FN = compute_eval_measures(ann_matrix, result)
        total_R += R
        total_P += P
        total_F += F1
        title = f'Eval: (N=%d, TP=%d, FP=%d, FN=%d, P=%.3f, R=%.3f, F=%.3f) - {sample_name}' % (result.shape[1], TP, FP, FN, P,R,F1)
        print(title)
    total_R /= count
    total_P /= count
    total_F /= count
    print(f'Total R: {total_R}')
    print(f'Total P: {total_P}')
    print(f'Total F: {total_F}')

In [ ]:
estimate_chords(sampler, model, device, batch_size=8)